In [1]:
from datetime import datetime
!pip install boto3
!pip install model-registry
import os
import boto3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
registeredmodel_name = "mnist"
version_name = "v1.nb"+datetime.now().strftime("%Y%m%d%H%M%S")
print(f"Will be using: {registeredmodel_name}:{version_name} in the remainder of this task")

Will be using: mnist:v1.nb20240325181713 in the remainder of this task


In [3]:
s3 = boto3.resource(
    service_name='s3',
    region_name=os.environ['AWS_DEFAULT_REGION'],
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    use_ssl=False,
    endpoint_url=os.environ['AWS_S3_ENDPOINT'],
    verify=False
)

In [4]:
bucket_name = os.environ['AWS_S3_BUCKET']
odh_secret_name = f'aws-connection-{bucket_name}'
in_bucket_path = version_name


In [5]:
in_bucket_target = f'registry/{in_bucket_path}/mnist.onnx'
print({in_bucket_target})

{'registry/v1.nb20240325181713/mnist.onnx'}


In [6]:
my_bucket = s3.Bucket(bucket_name)
my_bucket.upload_file("mnist.onnx", in_bucket_target)
print({my_bucket})

{s3.Bucket(name='ods-ci-ds-pipelines')}


In [7]:
my_bucket = s3.Bucket(bucket_name)
my_bucket.upload_file("mnist.onnx", in_bucket_target)

print("ls:")
for obj in my_bucket.objects.filter(Prefix='registry'):
    print(obj.key)

ls:
registry/
registry/v1.nb20240325180615/mnist.onnx
registry/v1.nb20240325181713/mnist.onnx


In [8]:
files = os.listdir(os.curdir) 
print(files)

['NotebookIntegration.ipynb', 'mnist.onnx', '.ipynb_checkpoints', 'lost+found', '.local', '.ipython', '.jupyter', '.cache']


In [9]:
from model_registry import ModelRegistry

registry = ModelRegistry(server_address="modelregistry-sample.model-registry-project.svc.cluster.local", port=9090, author="author")

In [10]:
rm = registry.register_model(registeredmodel_name,
                                in_bucket_target,
                                model_format_name="onnx",
                                model_format_version="1",
                                storage_key=odh_secret_name,
                                storage_path=in_bucket_path,
                                version=version_name,
                                description="demo20231121 e2e MNIST",
                                )

NameError: name 'full_bucket_target' is not defined

In [ ]:
print("RegisteredModel:")
print(registry.get_registered_model(registeredmodel_name))
print("ModelVersion:")
print(registry.get_model_version(registeredmodel_name, version_name))
print("ModelArtifact:")
print(registry.get_model_artifact(registeredmodel_name, version_name))